# visalise Session as Scroll using Ploty `Session.plotscroll()`

```bash
jupyter nbconvert "D:\OneDrive - Nexus365\Private_Dropbox\Projects\trialexp\notebooks\noncanonical\nb20221017_104300_session_plotscroll.ipynb" --to="python" --output-dir="D:\OneDrive - Nexus365\Private_Dropbox\Projects\trialexp\notebooks\noncanonical" --output="nb20221017_104300_session_plotscroll"
```

## Requirements

Plotly `conda install plotly`


    fftw-3.3.9                 |       h2bbff1b_1         672 KB
    icc_rt-2022.1.0            |       h6049295_2         6.5 MB
    numpy-1.23.1               |   py38h7a0a035_0          10 KB
    numpy-base-1.23.1          |   py38hca35cd5_0         5.0 MB
    plotly-5.9.0               |   py38haa95532_0         4.1 MB
    scikit-learn-1.1.2         |   py38hd77b12b_0         5.5 MB
    scipy-1.9.1                |   py38he11b74f_0        15.7 MB
    setuptools-63.4.1          |   py38haa95532_0         1.0 MB
    tenacity-8.0.1             |   py38haa95532_1          34 KB

### Imports

In [2]:
# allow for automatic reloading of classes and function when updating the code
%load_ext autoreload
%autoreload 2 

# Import Session and Experiment class with helper functions
from trialexp.process.data_import import *

### Variables

In [3]:
import pandas as pd

trial_window = [-2000, 6000] # in ms

# time limit around trigger to perform an event
# determine successful trials
timelim = [0, 2000] # in ms

# Digital channel nb of the pyphotometry device
# on which rsync signal is sent (from pycontrol device)
rsync_chan = 2

basefolder, _ = os.path.split(os.path.split(os.getcwd())[0])

# These must be absolute paths
# use this to use within package tasks files (in params)
tasksfile = os.path.join(basefolder,'params\\tasks_params.csv')
# use this to put a local full path
#tasksfile = -r'C:/.../tasks_params.csv' 

# photometry_dir = r'\\ettin\Magill_Lab\Julien\Data\head-fixed\test_folder\photometry'
photometry_dir = r'\\ettin\Magill_Lab\Julien\Data\head-fixed\kms_pyphotometry'
video_dir = r'\\ettin\Magill_Lab\Julien\Data\head-fixed\videos'


### Tasks
- A tasks definition file (.csv) contains all the information to perform the extractions of behaviorally relevant information from **PyControl** files, for each **task** file. It includes what are the **triggers** of different trial types, what **events** to extract (with time data), and what are events or printed lines that could be relevant to determine the **conditions** (e.g: free reward, optogenetic stimulation type, etc.)
- To analyze a new task you need to append task characteristics like **task** filename, **triggers**, **events** and **conditions**

In [4]:
tasks = pd.read_csv(tasksfile, usecols = [1,2,3,4], index_col = False)
tasks

,task,triggers,events,conditions
0,lick_go_nogo,go; nogo,bar; bar_off; spout,free_reward
1,lick_go_nogo_unconditionned,go; nogo,bar; bar_off; spout,free_reward
2,lick_go_nogo_unconditionned_opto,go; nogo,bar; bar_off; spout,free_reward
3,reaching_go_nogo,CS_Go; CS_NoGo,bar; bar_off; spout,error bar_off; reward bar_off; error free; rew...
4,reaching_go_nogo_jc,CS_plus; CS_minus,bar; bar_off; spout,error bar_off; reward bar_off; error free; rew...
5,reaching_go_nogo_opto_continuous,CS_Go; CS_NoGo,bar; bar_off; spout,s_nogo_sham; s_nogo_cs_onset; s_go_sham; s_go_...
6,reaching_go_nogo_opto_sinusoid,CS_Go; CS_NoGo,bar; bar_off; spout,s_nogo_sham; s_nogo_cs_onset_2; s_nogo_cs_onse...
7,reaching_go_nogo_opto_sinusoid_spout,CS_Go; CS_NoGo,bar; bar_off; spout,s_nogo_sham; s_nogo_cs_onset_2; s_nogo_cs_onse...
8,reaching_go_nogo_reversal,CS_Go; CS_NoGo,bar; bar_off; spout,error bar_off; reward bar_off; error free; rew...
9,reaching_go_nogo_reversal_incentive,CS_Go; CS_NoGo,bar; bar_off; spout,error bar_off; reward bar_off; error free; rew...


### Optional

Transfer Files from hierarchical folders by tasks to flat folders, for photometry and behaviour files

2m 13.9s

If we obtain list of files in source and dest at first and then only perform comparison on them,
This should be much faster

In [5]:
photo_root_dir = 'T:\\Data\\head-fixed\\pyphotometry\\data'
pycontrol_root_dir = 'T:\\Data\\head-fixed\\pycontrol'

root_folders = [photo_root_dir, pycontrol_root_dir]
horizontal_folder_pycontrol = 'T:\\Data\\head-fixed\\test_folder\\pycontrol'
horizontal_folder_photometry = 'T:\\Data\\head-fixed\\test_folder\\photometry'

copy_files_to_horizontal_folders(root_folders, horizontal_folder_pycontrol, horizontal_folder_photometry)

T:\Data\head-fixed\pyphotometry\data\cued_uncued_oct22\kms048-2022-10-11-102830.ppd
T:\Data\head-fixed\pyphotometry\data\cued_uncued_oct22\kms049-2022-10-14-114027.ppd
T:\Data\head-fixed\pyphotometry\data\cued_uncued_oct22\kms051-2022-10-11-114748.ppd
T:\Data\head-fixed\pyphotometry\data\cued_uncued_oct22\kms053-2022-10-10-170753.ppd
T:\Data\head-fixed\pyphotometry\data\go_rewarding_aversive\kms049-2022-10-12-120202.ppd
T:\Data\head-fixed\pyphotometry\data\go_rewarding_aversive\kms049-2022-10-12-125012.ppd
T:\Data\head-fixed\pyphotometry\data\go_rewarding_aversive\kms049-2022-10-13-123847.ppd
T:\Data\head-fixed\pyphotometry\data\pavlovian_nobar_nodelay\kms049-2022-09-23-124540.ppd
T:\Data\head-fixed\pyphotometry\data\pavlovian_nobar_nodelay\kms052-2022-09-23-140213.ppd
T:\Data\head-fixed\pycontrol\pavlovian_nobar_nodelay\00-2022-09-23-125237.txt
T:\Data\head-fixed\pycontrol\pavlovian_nobar_nodelay\kms049-2022-09-23-124542.txt
T:\Data\head-fixed\pycontrol\pavlovian_nobar_nodelay\kms052-

### Create an experiment object

This will include all the pycontrol files present in the folder_path directory (do not include subdirectories)

In [6]:
# Folder of a full experimental batch, all animals included

# Enter absolute path like this
# pycontrol_files_path = r'T:\Data\head-fixed\test_folder\pycontrol'

# or this if you want to use data from the sample_data folder within the package
pycontrol_files_path = os.path.join(basefolder,'sample_data/pycontrol')
pycontrol_files_path = r'T:\Data\head-fixed\kms_pycontrol'

# Load all raw text sessions in the indicated folder or a sessions.pkl file
# if already existing in folder_path
exp_cohort = Experiment(pycontrol_files_path)

# Only use if the Experiment cohort as been processed by trials before
# TODO: assess whether this can be removed or not
exp_cohort.by_trial = True

Saved sessions loaded from: sessions.pkl


### Perform extraction of behavioural information by trial

5m55.4s

In [7]:
# Process the whole experimental folder by trials
exp_cohort.process_exp_by_trial(trial_window, timelim, tasksfile, blank_spurious_event='spout', blank_timelim=[0, 65])

# Save the file as sessions.pkl in folder_path
# exp_cohort.save() # Do I need to save this???


No event  'spout'  found:  45 2022-04-27 15:57:48 pavlovian_nobar_nodelay
No event  'spout'  found:  42 2022-04-28 12:53:21 pavlovian_nobar_nodelay
No event  'spout'  found:  45 2022-05-06 12:27:18 pavlovian_nobar_nodelay
reaching_go_spout_cued_uncued 41 2022-05-11 17:30:36 25 29
reaching_go_spout_cued_uncued 42 2022-05-11 17:32:14 5 0
reaching_go_spout_cued_uncued 42 2022-05-11 17:47:30 38 3
reaching_go_spout_cued_uncued 43 2022-05-11 19:24:51 34 57
reaching_go_spout_cued_uncued 44 2022-05-11 19:24:51 22 37
reaching_go_spout_cued_uncued 45 2022-05-11 19:27:52 25 55
reaching_go_spout_cued_uncued 41 2022-05-12 15:28:29 33 105
reaching_go_spout_cued_uncued 42 2022-05-12 15:33:00 44 88
reaching_go_spout_cued_uncued 43 2022-05-12 16:32:18 51 61
reaching_go_spout_cued_uncued 44 2022-05-12 16:32:18 30 40
reaching_go_spout_cued_uncued 45 2022-05-12 16:37:28 11 0
reaching_go_spout_cued_uncued 45 2022-05-12 16:54:09 5 2
reaching_go_spout_cued_uncued 41 2022-05-16 13:21:21 3 0
reaching_go_spout_

### Match with photometry, videos, and DeepLabCut files

The following Warning : 

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads...

is due to rsync function for photometry-pycontrol alignment

2m10.9s


In [8]:
# Find if there is a matching photometry file and if it can be used:
# rsync synchronization pulses matching between behaviour and photometry
from copy import deepcopy

exp_cohort.match_to_photometry_files(photometry_dir, rsync_chan, verbose=False)

# Find matching videos
exp_cohort.match_sessions_to_files(video_dir, ext='mp4')

# FInd matching DeepLabCut outputs files
exp_cohort.match_sessions_to_files(video_dir, ext='h5')

exp_cohort.save()

exp_cohort_copy = deepcopy(exp_cohort)


c:\Users\phar0528\Anaconda3\envs\trialexp\lib\site-packages\sklearn\mixture\_base.py:131: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  cluster.KMeans(
d:\onedrive - nexus365\private_dropbox\projects\trialexp\trialexp\utils\rsync.py:100: RuntimeWarning: All-NaN slice encountered
  order_OK = ((np.nanmin(np.diff(cor_times_A)) > 0) and


## Visualise a session using Plotly

In [55]:

import plotly.graph_objects as go


%TODO

- markers
- drowdown to change time units
- express states by lines ... requires manual definition of each state

In [77]:
raw_symbols  = plotly.validators.scatter.marker.SymbolValidator().values
symbols = [raw_symbols[i+2] for i in range(0, len(raw_symbols), 12)]


session1 = exp_cohort.sessions[0]

fig = go.Figure()

keys = session1.times.keys()

for kind, k in enumerate(keys):

    sc = go.Line(x=session1.times[k]/1000, y=[k]
                 * len(session1.times[k]), name=k, mode='markers', marker_symbol=symbols[kind % 40])
    fig.add_trace(sc)

fig.update_xaxes(title='Time (s)')


fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    args=["type", "milliseconds"],
                    label="milliseconds",
                    method="restyle"
                ),
                dict(
                    args=["type", "seconds"],
                    label="seconds",
                    method="restyle"
                ),
                dict(
                    args=["type", "minutes"],
                    label="minutes",
                    method="restyle"
                )
            ]),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.04,
            xanchor="left",
            y=1.2,
            yanchor="top"
        ),
    ]
)

fig.update_layout(
    annotations=[
        dict(text="Time unit:", showarrow=False,
        x=-0, y=1.14, xref="paper", yref="paper", align="left")
    ]
)

fig.show()


In [89]:
exp_cohort.sessions[0].plot_session()


c:\Users\phar0528\Anaconda3\envs\trialexp\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


